<a href="https://colab.research.google.com/github/sota1111/DL_Zero3/blob/main/DL_Zero3_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os, sys
os.chdir('/content/drive/MyDrive/DL_Zero')

#!git clone https://github.com/sota1111/DL_Zero3.git

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'DL_Zero3' already exists and is not an empty directory.


In [ ]:
!git config --global user.email "sota.moro@gmail.com"
!git config --global user.name "sota1111"

# stage1 微分を自動で求める

## step1 箱としての変数

In [ ]:
class Variable:
  def __init__(self, data):
    self.data = data

In [ ]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


In [ ]:
x.data = np.array(2.0)
print(x.data)

2.0


In [ ]:
import numpy as np
x = np.array(1)
x.ndim

0

In [ ]:
x = np.array([1, 2, 3])
x.ndim

1

In [ ]:
x = np.array([[1,2,3],[4,5,6]])
x.ndim

2

## step2 変数を生み出す関数

In [ ]:
class Function:
  def __call__(self, input):
    x = input.data #データを取り出す
    y = x ** 2
    output = Variable(y) #Varibleとして取り出す
    return output

In [ ]:
x = Variable(np.array(10))
f = Function()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [ ]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    return output

  def forward(self, x):
    raise NotImplementedError()

In [ ]:
class Square(Function):
  def forward(self, x):
    return x ** 2

In [ ]:
x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


## step3 関数の連結

In [ ]:
class Exp(Function):
  def forward(self, x):
    return np.exp(x)

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


## step4 数値微分

In [ ]:
def numerical_diff(f, x, eps=1e-4):
  x0 = Variable(x.data - eps)
  x1 = Variable(x.data + eps)
  y0 = f(x0)
  y1 = f(x1)
  return (y1.data - y0.data)/(2*eps)

In [ ]:
f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(dy)

4.000000000004


In [ ]:
def f(x):
  A = Square()
  B = Exp()
  C = Square()
  return C(B(A(x)))

x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


## step5 バックプロパゲーションの理論

## step6 手作業によるバックプロパゲーション

In [ ]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None

In [ ]:
class Function:
  def __call__(self, input):
    x = input.data
    y = self.forward(x)
    output = Variable(y)
    self.input = input # inputされた変数を覚える
    return output

    def forward(self, x):
      raise NotImplementedError()

    def backward(self, gy):
      raise NotImplementedError()

In [ ]:
class Square(Function):
  def forward(self, x):
    y = x ** 2
    return y

  def backward(self, gy):
    x = self.input.data
    gx = 2 * x * gy
    return gx

In [ ]:
class Exp(Function):
  def forward(self, x):
    y = np.exp(x)
    return y
  
  def backward(self, gy):
    x = self.input.data
    gx = np.exp(x) * gy
    return gx

In [ ]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [ ]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


## step7 バックプロパゲーションの自動化

In [1]:
class Variable:
  def __init__(self, data):
    self.data = data
    self.grad = None
    self.creator = None

  def set_creator(self, func):
    self.creator = func